In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import json
from tqdm.notebook import tqdm

from wc_cv.src.utils import grainPreprocess, grainShow, grainMark, grainDraw, grainApprox, grainStats, grainMorphology,grainGenerate

from scipy import stats

# Paper angles

In [ ]:
def angles_approx_paper(data_1, data_2, save_name, step, N, M, indices_1, indices_2, save=False, font_size=20,scatter_size=20, axes_num=3, title=None, iter_index = None):
    #
    # вывод распределения углов для всех фотографий одного образца
    #

    alloys_indices=range(len(data_1))

    plt.rcParams['font.size'] = '15'
    
    fig, axes = plt.subplots(1,axes_num,figsize=(N, M))
    # fig, axes = plt.subplots(1,3,figsize=(N, M))

    markers = ['v', 's', 'D', 'd', 'p', '*','P']
    colors_1 = ['orange', 'red','blue','green', 'indigo', 'brown']
    colors_2 = ['olive', 'cyan','pink','purple', 'blue', 'lime']
    
#     colors_1 = ['orange', 'red','blue','green', 'indigo', 'brown']
#     colors_2 = ['olive', 'pink','pink','purple', 'blue', 'lime']
    
    sub_titles = ['Large grain','Medium grain ','Small grain' ]
    
    names_dict = {
                    'Ultra_Co6_2':'Large',
                    'Ultra_Co11':'Medium',
                    'Ultra_Co25':'Small',
                 }

    legends=[]
    metrics_list = []
    for k,(i,j) in enumerate(zip(indices_1, indices_2)):
        legends.append([])
        
        y_original=np.array(data_1[i]['density_curve_scatter'][1])
        y_fake=np.array(data_2[j]['density_curve_scatter'][1])
        w_dist = stats.wasserstein_distance(y_fake, y_original)
        
        tv_dist=0
        
        gauss_real=data_1[i]['gauss_approx_data']
        gauss_fake=data_2[j]['gauss_approx_data']
        
        mus_m = (np.array(gauss_fake['mus']) - np.array(gauss_real['mus']) )/np.array(gauss_real['mus']) 
        sigmas_m = (np.array(gauss_fake['sigmas']) - np.array(gauss_real['sigmas']))/np.array(gauss_real['sigmas'])
        amps_m = (np.array(gauss_fake['amps']) - np.array(gauss_real['amps']))/np.array(gauss_real['amps'])
        
        # metrics = f'\ntv_dist={tv_dist:.3f}\nw_dist={w_dist:.5f}\nw_gauss_dist={w_gauss_dist:.5f}\nmus_diff={mus_diff[0]:.2f};{mus_diff[1]:.2f}\nsigmas_diff={sigmas_diff[0]:.2f};{sigmas_diff[1]:.2f}'
        metrics = f'{w_dist*coef:2.3f} {mus_m[0]*100:.2f} {mus_m[1]*100:.2f} {sigmas_m[0]*100:.2f} {sigmas_m[1]*100:.2f} {amps_m[0]*100:.2f} {amps_m[1]*100:.2f}' 
        # print(metrics)
        # metrics = ''
        
        metrics_list.append(metrics)
        
        legends[k].append(names_dict[data_1[i]['name']]+ f" real")
        legends[k].append(names_dict[data_2[j]['name']]+ f" fake")

    for j,i in enumerate(indices_1):
        axes[j].plot(data_1[i]['gauss_approx_plot'][0], 
                 data_1[i]['gauss_approx_plot'][1],
                 color=colors_1[i])
    
    for j,i in enumerate(indices_2):
        axes[j].plot(data_2[i]['gauss_approx_plot'][0], 
                 data_2[i]['gauss_approx_plot'][1],
                 color=colors_2[i])

    for j,i in enumerate(indices_1):
        marker = markers[i]
        axes[j].scatter(data_1[i]['density_curve_scatter'][0],data_1[i]['density_curve_scatter'][1],  marker=marker,color=colors_1[i],s=scatter_size)
        if j==0:
            axes[j].set_ylabel('p(x)', fontsize=font_size)
        if iter_index==2:
            axes[j].set_xlabel('Angles in Co binder, deg.', fontsize=font_size-3)

    for j,i in enumerate(indices_2):
        marker = markers[i]
        axes[j].scatter(data_2[i]['density_curve_scatter'][0],data_2[i]['density_curve_scatter'][1],  marker=marker,color=colors_2[i],s=scatter_size)
        if j==0:
            axes[j].set_ylabel('p(x)', fontsize=font_size)
        if iter_index==0:
            axes[j].set_xlabel('Angles in Co binder, deg.', fontsize=font_size-3)

    x = [0,60,120,180,240,300,360]
    
    for j in range(len(axes)):
        axes[j].set_xticks(x, x)
        axes[j].legend(legends[j],loc='upper right')
        axes[j].set_ylim(0,0.07)

    if save:
        plt.savefig(f'распределение_углов_{save_name}_шаг_{step}.png', bbox_inches = "tight" )
    
    plt.show()
    
    return metrics_list

In [ ]:
# file_name_1='data/generated_data/o_bc_left_step_5_angles.json'
file_name_1 = 'data/generated_data/bc_left_(1536,1536,1)_preprocess_step_5_angles.json'

file_name_2_1='data/generated_data/o_bc_left_4x_768_360_768_median_generated_step_5_angles.json'
file_name_2_2='data/generated_data/o_bc_left_4x_768_360_512_median_generated_step_5_angles.json'
file_name_2_3='data/generated_data/o_bc_left_4x_768_360_256_median_generated_step_5_angles.json'

# file_name_2_1='data/generated_data/o_bc_left_4x_768_360_768_median_based_Ultra_Co25_generated_step_5_angles.json'
# file_name_2_2='data/generated_data/o_bc_left_4x_768_360_512_median_based_Ultra_Co25_generated_step_5_angles.json'
# file_name_2_3='data/generated_data/o_bc_left_4x_768_360_256_median_based_Ultra_Co25_generated_step_5_angles.json'

titles = ['768x768', '512x512','256x256']


names = [file_name_2_1, file_name_2_2, file_name_2_3]
# names = [file_name_2_1]

# N = 20*2/3

# N = 16
# M = 5
# N = 16*2/3


# 3 files
N = 16
M = 4

step=5

coef=1000
    
save = True
# save = False

data_1 = open(file_name_1,encoding='utf-8')
data_1 = json.load(data_1)['data']

metrics_all=[]

columns = f'w_dist*{coef} mu_1,% mu_2,%  sigma_1,% sigma_2,% amp_1,% amp_2,%'

for i,file_name_2 in enumerate(names):

    save_name=file_name_2.split('/')[-1]

    indices_1=[3,0, 2]
    indices_2=[2, 0, 1]
    
    # indices_1=[3,0]
    # indices_2=[1, 0]

    data_2 = open(file_name_2,encoding='utf-8')
    data_2 = json.load(data_2)

    # print(columns)
    metrics = angles_approx_2(data_1,data_2,save_name, step, N, M, indices_1=indices_1,indices_2=indices_2,
                              save=save, axes_num=3, title=titles[i], iter_index=i)
    metrics_all.extend(metrics)
    

print(columns)
for m in metrics_all:
    print(m)